# Анализ базы данных сервиса для чтения книг по подписке

Необходимо  проанализировать базу данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

Цель - сформулировать ценностное предложение для нового продукта.

## Исследование таблиц

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import create_engine

Создадим подключение к базе данных.

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем первые строки каждой из таблиц.

In [3]:
# функция для вывода резултатов SQL-запроса

def sql (query):
  return pd.io.sql.read_sql(query, con = engine)  

Таблица books

In [4]:
query = """
        SELECT *
        FROM books
        LIMIT 5
        """
sql (query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


    Таблица books
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors

In [5]:
query = """
        SELECT *
        FROM authors
        LIMIT 5
        """
sql (query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


      Таблица authors  
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers

In [6]:
query = """
        SELECT *
        FROM publishers
        LIMIT 5
        """
sql (query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


      Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings

In [7]:
query = """
        SELECT *
        FROM ratings
        LIMIT 5
        """
sql (query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


      Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews

In [8]:
query = """
        SELECT *
        FROM reviews
        LIMIT 5
        """
sql (query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


      Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Количество  книг, вышедших после 1 января 2000 года

In [9]:
query = """
        SELECT COUNT(book_id) as books_amount
        FROM books
        WHERE publication_date > '2000-01-01'
        """

sql(query)

,books_amount
0,819


После первого января было выпущенно 819 книг.

##  Количество обзоров и средняя оценка для каждой книги

In [10]:
query = """
        SELECT b.title, COUNT(DISTINCT(re.review_id)) as amount_reviews,AVG(r.rating) as avg_rating
        FROM books b 
        LEFT JOIN ratings r on b.book_id = r.book_id
        LEFT JOIN reviews re on b.book_id = re.book_id
        GROUP BY b.book_id
        ORDER by amount_reviews DESC
        """

sql(query)

,title,amount_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Для 1000 книг мы вывели количество обзоров и среднюю оценку читателей. По количеству обзоров лидирует Twilight (Twilight #1)

##  Издательство, выпустившее наибольшее число книг толще 50 страниц

In [11]:
query = """
        SELECT p.publisher, COUNT(b.book_id) as amount_books
        FROM books b 
        JOIN publishers p on b.publisher_id = p.publisher_id
        WHERE b.num_pages > 50
        GROUP BY p.publisher
        ORDER by amount_books DESC
        LIMIT 1
        """
sql(query)

,publisher,amount_books
0,Penguin Books,42


Издательство с наибольшим количеством книг  - Penguin Books. Оно выпустило 42 книги  толще 50 страниц.

##  Автор  с самой высокой средней оценкой книг  (учитываем  только книги с 50 и более оценками)

In [15]:
query = """
        WITH popular_books AS
        (SELECT b.title, MAX(a.author) as author,b.author_id as a_id, AVG(r.rating) as avg_rating, COUNT(r.rating)
        FROM books b 
        JOIN ratings r on b.book_id = r.book_id
        JOIN authors a on b.author_id = a.author_id
        GROUP BY b.book_id
        HAVING COUNT(r.rating)>=50)
        SELECT MAX(popular_books.author), AVG(popular_books.avg_rating)
        FROM popular_books
        GROUP BY popular_books.a_id
        ORDER BY AVG(popular_books.avg_rating) desc
        LIMIT 1
        """
sql(query)

,max,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автором с самой высокой средней оценкой его популярных книг является J.K. Rowling(Mary GrandPré)

## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = """
        SELECT AVG(A.count_review) as avg_amount_reviews
        FROM  (SELECT COUNT(r.review_id) as count_review 
                FROM reviews r
                WHERE r.username IN (SELECT    username
                                     FROM      ratings
                                     GROUP BY  username
                                     HAVING    COUNT(ratings) > 50)
                GROUP BY  r.username) as A                   
        """
sql (query)

,avg_amount_reviews
0,24.333333


Среднее количество обзоров от пользователей, которые часто ставят оценки (50 и более раз) равно 24 шт.

## Выводы

- После 1 января 2000 года было выпущено 819 книг.
- По  количеству обзоров на первом месте книга "Twilight (Twilight #1)" со средним рейтингом 3.6.
- Издательство, выпустившее наибольшее количество книг -  "Penguin Books" (42 книги толде 50 стр.)
- Автор с самой высокой средней оценкой его популярных книг - J.K. Rowling(Mary GrandPré).
- Среднее количество обзоров от пользователей, поставивших более 50 оценок, - 24.

Для нового продукта необходимо акцентировать внимание на популярных книгах с высоким количеством обзоров. Так для наиболее популярных книг количество обзоров от 370 до 1120.